In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan_preLSTM2.model import DATGAN
#from modules.datgan_no_att.model import DATGAN
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("work_status", "hh_income"),
        ("education_level", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/DATGAN_TEST/'

In [10]:
datgan = DATGAN(continuous_columns, max_epoch=1000, steps_per_epoch=2000, batch_size=200, output=output_folder, gpu=0)

In [ ]:
datgan.fit(df, graph)

[0924 10:10:38 @model.py:873] Found an already existing model. Loading it!
[0924 10:10:38 @model.py:885] Preprocessed data have been loaded!
[0924 10:10:38 @input_source.py:222] Setting up the queue 'QueueInput_1/input_queue' for CPU prefetching ...
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
-> Creating cell for age (in-edges: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0924 10:10:38 @registry.py:126] gen/LSTM/00/FC input: [200, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0924 10:10:38 @registry.py:134] gen/LSTM/00/FC output: [200, 100]
[0924 10:10:38 @registry.py:126] gen/LSTM/00/FC2 input: [200, 100]
[0924 10:10:38 @registry.py:134] gen/LSTM/00/FC2 output: [200, 1]
[0924 10:10:38 @registry.py:126] gen/LSTM/01/FC input

[0924 10:10:39 @registry.py:134] gen/LSTM/11/FC_lstm_state_0 output: [200, 100]
[0924 10:10:39 @registry.py:126] gen/LSTM/11/FC_lstm_state_1 input: [200, 200]
[0924 10:10:39 @registry.py:134] gen/LSTM/11/FC_lstm_state_1 output: [200, 100]
[0924 10:10:39 @registry.py:126] gen/LSTM/12/FC input: [200, 100]
[0924 10:10:39 @registry.py:134] gen/LSTM/12/FC output: [200, 100]
[0924 10:10:39 @registry.py:126] gen/LSTM/12/FC2 input: [200, 100]
[0924 10:10:39 @registry.py:134] gen/LSTM/12/FC2 output: [200, 3]
[0924 10:10:39 @registry.py:126] gen/LSTM/12/FC3 input: [200, 3]
[0924 10:10:39 @registry.py:134] gen/LSTM/12/FC3 output: [200, 100]
-> Creating cell for hh_size (in-edges: 3)
[0924 10:10:39 @registry.py:126] gen/LSTM/13/FC_inputs input: [200, 300]
[0924 10:10:39 @registry.py:134] gen/LSTM/13/FC_inputs output: [200, 100]
[0924 10:10:39 @registry.py:126] gen/LSTM/13/FC_attentions input: [200, 300]
[0924 10:10:39 @registry.py:134] gen/LSTM/13/FC_attentions output: [200, 100]
[0924 10:10:39 @r

[0924 10:10:39 @registry.py:126] discrim/dis_fc_top input: [200, 110]
[0924 10:10:39 @registry.py:134] discrim/dis_fc_top output: [200, 1]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


[0924 10:10:43 @logger.py:83] Existing log file '../output/Chicago/DATGAN_TEST/logs\log.log' backuped to '../output/Chicago/DATGAN_TEST/logs\log.log.0924-101043'
[0924 10:10:43 @logger.py:90] Argv: D:\Users\glede\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py -f C:\Users\glede\AppData\Roaming\jupyter\runtime\kernel-62c97084-4acd-4fef-b001-696bf564690f.json


[0924 10:10:43 @model_utils

[0924 10:10:43 @base.py:209] Setup callbacks graph ...






[0924 10:10:44 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0924 10:10:44 @summary.py:93] Summarizing collection 'summaries' of size 10.

[0924 10:10:44 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.



[0924 10:10:45 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0924 10:10:47 @base.py:236] Initializing the session ...
[0924 10:10:47 @sessinit.py:114] Restoring checkpoint from ../output/Chicago/DATGAN_TEST/model\model-20000 ...
INFO:tensorflow:Restoring parameters from ../output/Chicago/DATGAN_TEST/model\model-20000
[0924 10:10:47 @base.py:243] Graph Finalized.


[0924 10:10:47 @concurrency.py:38] Starting EnqueueThread QueueInput_1/input_queue ...
[0924 10:10:47 @steps.py:124] Start training with global_step=20000
[0924 10:10:48 @monitor.py:347] Found

100%|################################################################################7|1995/2000[00:39<00:00,54.61it/s]

...

[0924 13:12:41 @base.py:285] Epoch 303 (global_step 606000) finished, time:37.1 seconds.
[0924 13:12:41 @saver.py:79] Model saved to ../output/Chicago/DATGAN_TEST/model\model-606000.
[0924 13:12:42 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.94
[0924 13:12:42 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.15
[0924 13:12:42 @monitor.py:467] GAN_loss/discrim/loss: 0.64456
[0924 13:12:42 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.3764
[0924 13:12:42 @monitor.py:467] GAN_loss/gen/klloss: 0.22923
[0924 13:12:42 @monitor.py:467] GAN_loss/gen/loss: 1.1472
[0924 13:12:42 @monitor.py:467] QueueInput/queue_size: 1.8367e-38
[0924 13:12:42 @monitor.py:467] QueueInput_1/queue_size: 50
[0924 13:12:42 @base.py:275] Start Epoch 304 ...



100%|#################################################################################|2000/2000[00:37<00:00,53.13it/s]

[0924 13:13:19 @base.py:285] Epoch 304 (global_step 608000) finished, time:37.6 seconds.
[0924 13:13:19 @saver.py:79] Model saved to ../output/Chicago/DATGAN_TEST/model\model-608000.


[0924 13:13:19 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.945
[0924 13:13:19 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.15
[0924 13:13:19 @monitor.py:467] GAN_loss/discrim/loss: 0.6372
[0924 13:13:19 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.4141
[0924 13:13:19 @monitor.py:467] GAN_loss/gen/klloss: 0.24921
[0924 13:13:19 @monitor.py:467] GAN_loss/gen/loss: 1.1649
[0924 13:13:19 @monitor.py:467] QueueInput/queue_size: 1.8367e-38
[0924 13:13:19 @monitor.py:467] QueueInput_1/queue_size: 50
[0924 13:13:19 @base.py:275] Start Epoch 305 ...


  9%|#######7                                                                          |189/2000[00:03<00:34,53.03it/s]

In [ ]:
datgan.save('trained', force=True)